In [1]:
from src.api import DataAPI , ModelPredictor , ModelHiddenExtractor
from pymysql.err import OperationalError
try:
    DataAPI.update()
except Exception:
    pass
DataAPI.prepare_predict_data()

ModelHiddenExtractor.update_hidden('gru_day' , model_submodels=['best'])
ModelPredictor.update_factors()

Update Files
Sun Oct 27 18:37:57 2024 : Updated ~ DB_information\calendar.feather Done! Cost 0.02 Secs
Sun Oct 27 18:37:57 2024 : Updated ~ DB_information\description.feather Done! Cost 0.03 Secs
Sun Oct 27 18:37:57 2024 : Updated ~ DB_information\st.feather Done! Cost 0.02 Secs
Sun Oct 27 18:38:00 2024 : Updated ~ DB_information\industry.feather Done! Cost 2.75 Secs
Sun Oct 27 18:38:06 2024 : Updated ~ DB_information\concepts.feather Done! Cost 6.52 Secs
Sun Oct 27 18:38:08 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20241022.feather Done! Cost 0.05 Secs
Sun Oct 27 18:38:08 2024 : Updated ~ DB_models\risk_cov\2024\risk_cov.20241022.feather Done! Cost 0.03 Secs
Sun Oct 27 18:38:08 2024 : Updated ~ DB_models\risk_spec\2024\risk_spec.20241022.feather Done! Cost 0.02 Secs
Sun Oct 27 18:38:08 2024 : Updated ~ DB_models\longcl_exp\2024\longcl_exp.20241022.feather Done! Cost 0.10 Secs
Sun Oct 27 18:38:08 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20241023.feather Done! Cost 0.05

100%|██████████| 316/316 [00:00<00:00, 2084.67it/s]


--------------------------------------------------------------------------------


 99%|█████████▉| 94/95 [00:02<00:00, 32.68it/s] 


Finish model [gru_day] predicting!


 99%|█████████▉| 94/95 [00:03<00:00, 25.20it/s] 


Finish model [gruRTN_day] predicting!


 99%|█████████▉| 94/95 [00:01<00:00, 61.12it/s] 


Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


ModelPredictor(reg_model=RegisteredModel(name=gru_avg,type=swalast,num=all,alias=gru_day_V1))

In [1]:
from src.api import DataAPI , ModelPredictor

DataAPI.update()
DataAPI.prepare_predict_data()
ModelPredictor.update_factors()

Update Files
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\calendar.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\description.feather Done! Cost 0.03 Secs
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\st.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:48 2024 : Updated ~ DB_information\industry.feather Done! Cost 2.55 Secs
Thu Sep 19 00:49:53 2024 : Updated ~ DB_information\concepts.feather Done! Cost 5.54 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\longcl_exp\2024\longcl_exp.20240913.feather Done! Cost 0.10 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20240918.feather Done! Cost 0.05 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_cov\2024\risk_cov.20240918.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_spec\2024\risk_spec.20240918.feather Done! Cost 0.01 Secs
Thu Sep 19 00:49:57 2024 : Updated ~ DB_trade\day\2024\day.20240918.feather Done! Cost 0.05 Secs
Thu S

Predictor(reg_model=RegModel(name=gru_avg,type=swabest,num=all,alias=gru_day_V1))

In [8]:
from src.data.tushare.download.info import Calendar , pro
renamer = {'cal_date' : 'calendar' ,'is_open'  : 'trade'}
fields = None
df = pro.trade_cal(exchange='SSE').rename(columns=renamer)
#df = df.sort_values('calendar').reset_index(drop = True)
df

""


In [10]:
# DataAPI.prepare_predict_data()
ModelPredictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [2]:
from src.api import DataAPI , ModelPredictor
# DataAPI.prepare_predict_data()
md = ModelPredictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [ ]:
from src.api import ModelTestor
model = ModelTestor('ts_mixer')
model.try_forward()
model.try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
